In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier

data_folder = os.path.join("..", "data")
plots_folder = os.path.join("..", "plots")

# Load assets
assets = {}
for ticker in ["TSLA", "BND", "SPY"]:
    df = pd.read_csv(os.path.join(data_folder, f"{ticker}.csv"), parse_dates=[0], index_col=0)
    df["Daily_Return"] = df["Close"].pct_change()
    assets[ticker] = df

# Combine returns
returns_df = pd.concat([assets[t]["Daily_Return"] for t in assets], axis=1)
returns_df.columns = assets.keys()
returns_df.dropna(inplace=True)

# Expected returns & covariance
mu = mean_historical_return(pd.concat([assets[t]["Close"] for t in assets], axis=1))
S = CovarianceShrinkage(pd.concat([assets[t]["Close"] for t in assets], axis=1)).ledoit_wolf()

# Optimize portfolio
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
performance = ef.portfolio_performance(verbose=True)

# Plot Efficient Frontier
from pypfopt import plotting
ef = EfficientFrontier(mu, S)
fig, ax = plt.subplots(figsize=(8,6))
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.savefig(os.path.join(plots_folder, "task4_efficient_frontier.png"))
plt.show()

print("Optimal Weights:", cleaned_weights)
